In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My Drive/AA_PROJ_2

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/AA_PROJ_2


In [ ]:
import pandas as pd
import os
import re
from tqdm import tqdm
import pickle
import numpy as np
import librosa


In [ ]:
tracks = pd.read_csv("data/FMA/FMA/fma_metadata/tracks.csv", index_col=0, header=[0, 1])
tracks.head()

In [ ]:
tracksFiltered = tracks[[('set', 'split'),('set', 'subset'),('track', 'genre_top'), ('track', 'genres')]]
isSmallData = tracksFiltered[('set', 'subset')] == 'small' 

In [ ]:
genreNamesDF = pd.read_csv("data/FMA/FMA/fma_metadata/genres.csv", index_col=0, header=0)
genreDict = genreNamesDF["title"].squeeze().to_dict()

In [ ]:
filelist =[]
for root, dirs, files in os.walk("data/FMA/FMA/fma_small"):
	for file in files:
		filelist.append(os.path.join(root,file))
	
regex = re.compile(r'data\/FMA\/FMA\/fma_small\/\d{3}\/\d+.mp3')
filelist_filtered = list(filter(regex.search, filelist))

smallDataset.loc[id][('track', 'genre_top')]

In [ ]:
#------Waveform-------
wf_x_train = []
wf_x_test = []
wf_x_valid = []

#------Spectrogram-------
sp_x_train = []
sp_x_test = []
sp_x_valid = []


#label 

y =[]

number=0

smallDataset = tracksFiltered[isSmallData][[('set', 'split'),('set', 'subset'),('track', 'genre_top')]]
for fileName in tqdm(filelist_filtered[3000:]):

  number = number +1
  id = int(re.compile(r'\d+(?=.mp3)').search(fileName).group())
  print(fileName)
  try:
    wf_train, sr_train = librosa.load(fileName, offset=0, duration=9)
    wf_test, sr_test = librosa.load(fileName, offset=9, duration=9)
    wf_valid, sr_valid = librosa.load(fileName, offset=18, duration=9)

    sp_train = librosa.feature.melspectrogram(y=wf_train, sr=sr_train)
    sp_test = librosa.feature.melspectrogram(y=wf_test, sr=sr_test)
    sp_valid = librosa.feature.melspectrogram(y=wf_valid, sr=sr_valid)

    isWaveformValid = wf_train.shape[0] == 198450 and wf_test.shape[0] == 198450 and wf_valid.shape[0] == 198450
    isSpectrogramValid = sp_train.shape == (128, 388) and sp_test.shape == (128, 388) and sp_valid.shape == (128, 388)


    if(isWaveformValid and isSpectrogramValid): 
      #**************************************************************
      #*                        Waveforms                           *
      #**************************************************************

      # ---- waveform_train----
      wf_x_train.append(wf_train.reshape(-1,1))
      
      # ---- waveform_test-----
      wf_x_test.append(wf_test.reshape(-1,1))

      # ---- waveform_valid----
      wf_x_valid.append(wf_valid.reshape(-1,1))



      #**************************************************************
      #*                        Spectrograms                        *
      #**************************************************************

      # ---- spectrogram_train----
      sp_x_train.append(sp_train)
      
      # ---- spectrogram_test----
      sp_x_test.append(sp_test)

      # ---- spectrogram_valid----
      sp_x_valid.append(sp_valid)


      ## Label
      y.append(smallDataset.loc[id][('track', 'genre_top')])
  
    if number == 1000:
      print("dumping files")
      pickle_out = open("Pickled_Data/Final/SP/wf_x_train.pickle","ab") 
      pickle.dump(np.array(wf_x_train),pickle_out)
      pickle_out.close()

      pickle_out = open("Pickled_Data/Final/SP/wf_x_test.pickle","ab") 
      pickle.dump(np.array(wf_x_test),pickle_out)
      pickle_out.close()

      pickle_out = open("Pickled_Data/Final/SP/wf_x_valid.pickle","ab") 
      pickle.dump(np.array(wf_x_valid),pickle_out)
      pickle_out.close()

      pickle_out = open("Pickled_Data/Final/SP/sp_x_train.pickle","ab") 
      pickle.dump(np.array(sp_x_train),pickle_out)
      pickle_out.close()

      pickle_out = open("Pickled_Data/Final/SP/sp_x_test.pickle","ab") 
      pickle.dump(np.array(sp_x_test),pickle_out)
      pickle_out.close()

      pickle_out = open("Pickled_Data/Final/SP/sp_x_valid.pickle","ab") 
      pickle.dump(np.array(sp_x_valid),pickle_out)
      pickle_out.close()

      pickle_out = open("Pickled_Data/Final/SP/y.pickle","ab") 
      pickle.dump(np.array(y),pickle_out)
      pickle_out.close()

      #------Waveform-------
      wf_x_train = []
      wf_x_test = []
      wf_x_valid = []
      #------Spectrogram-------
      sp_x_train = []
      sp_x_test = []
      sp_x_valid = []
      #label 
      y =[]
      number = 0
    

  except:
    print("bad format")

In [ ]:
#------Waveform-------
np_wf_x_train = np.array_split(np.array(wf_x_train), 10)

np_wf_x_test = np.array_split(np.array(wf_x_test), 10)

np_wf_x_valid = np.array_split(np.array(wf_x_valid), 10)

#------Spectrogram-------
np_sp_x_train = np.array_split(np.array(sp_x_train), 10)

np_sp_x_test = np.array_split(np.array(sp_x_test), 10)

np_sp_x_valid = np.array_split(np.array(sp_x_valid), 10)


#------Label------------
np_y = np.array_split(np.array(y), 10)

In [ ]:
#------Waveform-------
del wf_x_train 
 
del wf_x_test
 
del wf_x_valid
 
 #------Spectrogram-------
del sp_x_train
 
del sp_x_test
 
del sp_x_valid

#------Label-------------

del y

#Pickle

WaveForm

In [ ]:
pickle_out = open("Pickled_Data/Final/WF/np_wf_x_train.pickle","wb")
for split in np_wf_x_train:
  pickle.dump(split,pickle_out)
pickle_out.close()


In [ ]:
pickle_out = open("Pickled_Data/Final/WF/np_wf_x_test.pickle","wb")
for split in np_wf_x_test:
  pickle.dump(split,pickle_out)
pickle_out.close()


In [ ]:
pickle_out = open("Pickled_Data/Final/WF/np_wf_x_valid.pickle","wb")
for split in np_wf_x_valid:
  pickle.dump(split,pickle_out)
pickle_out.close()



Spectrogram

In [ ]:
pickle_out = open("Pickled_Data/Final/SP/np_sp_x_train.pickle","wb")
for split in np_sp_x_train:
  pickle.dump(split,pickle_out)
pickle_out.close()


In [ ]:
pickle_out = open("Pickled_Data/Final/SP/np_sp_x_test.pickle","wb")
for split in np_sp_x_test:
  pickle.dump(split,pickle_out)
pickle_out.close()


In [ ]:
pickle_out = open("Pickled_Data/Final/SP/np_sp_x_valid.pickle","wb")
for split in np_sp_x_valid:
  pickle.dump(split,pickle_out)
pickle_out.close()


label


In [ ]:
pickle_out = open("Pickled_Data/Final/np_y.pickle","wb")
for split in np_y:
  pickle.dump(split,pickle_out)
pickle_out.close()

#Alternative

In [ ]:
import pandas as pd
import os
import re
from tqdm import tqdm
import pickle
import numpy as np
import librosa


In [ ]:
tracks = pd.read_csv("data/FMA/FMA/fma_metadata/tracks.csv", index_col=0, header=[0, 1])


In [ ]:
tracksFiltered = tracks[[('set', 'split'),('set', 'subset'),('track', 'genre_top'), ('track', 'genres')]]
isSmallData = tracksFiltered[('set', 'subset')] == 'small' 

In [ ]:
genreNamesDF = pd.read_csv("data/FMA/FMA/fma_metadata/genres.csv", index_col=0, header=0)
genreDict = genreNamesDF["title"].squeeze().to_dict()

In [ ]:
filelist =[]
y=[]
smallDataset = tracksFiltered[isSmallData][[('set', 'split'),('set', 'subset'),('track', 'genre_top')]]
isMp3 = re.compile(r'\d+.mp3')

id = re.compile(r'\d+(?=.mp3)')

for root, dirs, files in os.walk("data/FMA/FMA/fma_small"):
	for file in files:
		if isMp3.search(file):
			filelist.append(file)
			num = int(id.search(file).group())
			y.append(smallDataset.loc[num][('track', 'genre_top')])
	


In [ ]:
file_list_out = open("Pickled_Data/simple_pickle/file_list.pickle","wb") 
pickle.dump(filelist,file_list_out)
file_list_out.close()
y_out = open("Pickled_Data/simple_pickle/labels.pickle","wb") 
pickle.dump(y,y_out)
y_out.close()

In [ ]:
filelist =[]
y=[]
smallDataset = tracksFiltered[isSmallData][[('set', 'split'),('set', 'subset'),('track', 'genre_top')]]
isMp3 = re.compile(r'\d+.mp3')

id = re.compile(r'\d+(?=.mp3)')

for root, dirs, files in os.walk("data/FMA/FMA/fma_small"):
	for file in files:
		if isMp3.search(file):
			print(file)
			num = int(id.search(file).group())
			i =0
			i =0

In [ ]:
print("dumping FINAL files")
pickle_out = open("Pickled_Data/Final/SP/np_wf_x_train.pickle","ab") 
pickle.dump(np.array(wf_x_train),pickle_out)
pickle_out.close()

pickle_out = open("Pickled_Data/Final/SP/wf_x_test.pickle","ab") 
pickle.dump(np.array(wf_x_test),pickle_out)
pickle_out.close()

pickle_out = open("Pickled_Data/Final/SP/wf_x_valid.pickle","ab") 
pickle.dump(np.array(wf_x_valid),pickle_out)
pickle_out.close()

pickle_out = open("Pickled_Data/Final/SP/sp_x_train.pickle","ab") 
pickle.dump(np.array(sp_x_train),pickle_out)
pickle_out.close()

pickle_out = open("Pickled_Data/Final/SP/sp_x_test.pickle","ab") 
pickle.dump(np.array(sp_x_test),pickle_out)
pickle_out.close()

pickle_out = open("Pickled_Data/Final/SP/sp_x_valid.pickle","ab") 
pickle.dump(np.array(sp_x_valid),pickle_out)
pickle_out.close()

pickle_out = open("Pickled_Data/Final/SP/y.pickle","ab") 
pickle.dump(np.array(y),pickle_out)
pickle_out.close()

dumping FINAL files
